In [35]:
import urllib
from bs4 import BeautifulSoup
from datetime import datetime

In [106]:
year = datetime.today().strftime("%Y")
start = datetime(2020,9,8)
week = str(round(((datetime.today() - start).days+2)/7))

In [107]:
def crawler(url):
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [108]:
fte_url = 'https://projects.fivethirtyeight.com/'+year+'-nfl-predictions/games/'
fte_soup = crawler(fte_url)
vegas_url = 'https://www.vegasinsider.com/nfl/matchups/matchups.cfm/week/'+week+'/season/'+year
vegas_soup = crawler(vegas_url)

In [39]:
def team_match(x):
    if 'jets' in x.lower():
        return 'Jets'
    if 'indiana' in x.lower():
        return 'Colts'
    if 'denver'in x.lower():
        return 'Broncos'
    if 'chica'in x.lower():
        return 'Bears'
    if 'orlean'in x.lower():
        return 'Saints'
    if 'arizon' in x.lower():
        return 'Cardinals'
    if 'carolin' in x.lower():
        return 'Panthers'
    if 'detroit' in x.lower():
        return 'Lions'
    if 'cinci' in x.lower():
        return 'Bengals'
    if 'jackson'in x.lower():
        return 'Jaguars'
    if 'dallas' in x.lower():
        return 'Cowboys'
    if 'cleve' in x.lower():
        return 'Browns'
    if 'houst'in x.lower():
        return 'Texans'
    if 'minne' in x.lower():
        return 'Vikings'
    if 'seat' in x.lower():
        return 'Seahwaks'
    if 'miami' in x.lower():
        return 'Dolphins'
    if 'tampa' in x.lower():
        return 'Buccaneers'
    if 'charg' in x.lower():
        return 'Chargers'
    if 'pitt' in x.lower():
        return 'Steelers'
    if 'tenn' in x.lower():
        return 'Titans'
    if 'balti' in x.lower():
        return 'Ravens'
    if 'washing' in x.lower():
        return 'Washington'
    if 'rams' in x.lower():
        return 'Rams'
    if 'giant' in x.lower():
        return 'Giants'
    if 'kansas' in x.lower():
        return 'Chiefs'
    if 'england' in x.lower():
        return 'Patriots'
    if 'buff' in x.lower():
        return "Bills"
    if 'vegas' in x.lower():
        return 'Raiders'
    if 'franc' in x.lower():
        return '49ers'
    if 'phila' in x.lower():
        return 'Eagles'
    if 'green' in x.lower():
        return 'Packers'
    if 'atlan' in x.lower():
        return 'Falcons'
    else:
        return 'ERROR'

In [109]:
elo_favorites = []
elo_underdogs = []
elo_spreads = []
current_week_tag_list = fte_soup.find("div", class_="days").find_all("tr")


for tag in current_week_tag_list:
    if tag.contents[0].get('class') != ['th', 'time']:
        if len(tag.contents[2].text) > 1:
            elo_favorites.append(tag.contents[1].text.strip())
            try:
                elo_spreads.append(float(tag.contents[2].text[2:]))
            except:
                elo_spreads.append(float(0))
        else:
            elo_underdogs.append(tag.contents[1].text.strip())
    else:
        pass

elo_tuple = list(zip(elo_favorites, elo_underdogs, elo_spreads))

In [110]:
vegas_favorites = []
vegas_underdogs = []
vegas_spreads = []

current_week_game_tags = vegas_soup.find_all("div",class_='SLTables1')[1].find_all("div",class_='SLTables1')

for game_tag in current_week_game_tags:
    for row_tag in game_tag.find_all("tr")[4:6]:
        if '-' in row_tag.find_all('td')[4].text:
            vegas_favorites.append(team_match(row_tag.find('a').text))
            vegas_spreads.append(float(row_tag.find_all('td')[4].text[1:]))
        elif 'PK' in row_tag.find_all('td')[4].text:
            vegas_favorites.append(team_match(row_tag.find('a').text))
            vegas_spreads.append(float(0))
        else:
            vegas_underdogs.append(team_match(row_tag.find('a').text))

vegas_tuple = list(zip(vegas_favorites,vegas_underdogs,vegas_spreads))

In [ ]:
# need to update the plays to describe other team and spreads

In [120]:
teams_to_bet = []
elo_to_vegas_abs_diffs = []

for elo_tup in elo_tuple:
    for vegas_tup in vegas_tuple:
        if elo_tup[0] not in vegas_tup:
            pass
        elif elo_tup[0] == vegas_tup[0] and elo_tup[1] == vegas_tup[1]:
            elo_to_vegas_diff = elo_tup[2] - vegas_tup[2]
            if elo_to_vegas_diff <= 0:
                teams_to_bet.append(elo_tup[1])
            else:
                teams_to_bet.append(elo_tup[0])
            elo_to_vegas_abs_diffs.append(abs(elo_to_vegas_diff))
        elif elo_tup[0] == vegas_tup[1] and elo_tup[1] == vegas_tup[0]:
            elo_to_vegas_diff = elo_tup[2] + vegas_tup[2]
            teams_to_bet.append(elo_tup[0])
            elo_to_vegas_abs_diffs.append(abs(elo_to_vegas_diff))

plays = list(zip(teams_to_bet,elo_to_vegas_abs_diffs))
plays.sort(key=lambda tup: tup[1], reverse=True)


for i in range(len(plays)):
    if plays[i][1] > 1.5:
        print('{} is the number {} play this week with a {}pt spread difference'.format(plays[i][0],
                                                                        i+1,plays[i][1]), sep='\n')

Jets is the number 1 play this week with a 3.5pt spread difference
Bengals is the number 2 play this week with a 2.0pt spread difference
Dolphins is the number 3 play this week with a 2.0pt spread difference
Broncos is the number 4 play this week with a 2.0pt spread difference
Titans is the number 5 play this week with a 2.0pt spread difference
